# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [79]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

We consider the **standard OLG-model** where:

We assume that utility is given by $u(c)$ with constant relative risk aversion (CRRA utility function): 

\begin{equation}
  u(c)=\begin{cases}
    \frac{c^{1-\sigma}-1}{1-\sigma} , & \sigma ≠ 1 \\
    log c, & \sigma = 1
  \end{cases}
\end{equation}

Individuals live for two periods, t = 1,2. $L_t$ individuals are born in period t and we assume that the population grows with a constant rate, n: 

$$ L_t = L_{t-1}(1+n) $$

agents derive utility while alive: 

$$ U_t = u(c_{1t})+\frac{1}{1+\rho} u(c_{2t}) $$

Agtens' budget constraint in each period is given by: 

$$ c_{1t} + s_t = w_t $$

$$ c_{2t+1} = (1+r_{t+1}) s_t $$

This implies that agents' lifetime budget constraint is given by: 

$$ c_{1t} + \frac{c_{2t+1}}{1+r_{t+1}} = w_t $$



# Steady state

## Analytical solution

Every agtens born at time t maximize utility subject to their lifetime budget constraint:

First we define all **symbols**:

In [136]:
c_1t = sm.symbols('c_1t')
c_2t = sm.symbols('c_2t+1')
rho = sm.symbols('rho')
u_1 = sm.symbols('u_1')
u_2 = sm.symbols('u_2')
sigma = sm.symbols('sigma')
w = sm.symbols('w_t')
r_t1 = sm.symbols('r_t+1')
lamb = sm.symbols('lambda')
s = sm.symbols('s_t')

0

In [109]:
# Define equations
budget_c = sm.Eq(c_1t + c_2t/(1+r_t1), w)

u_c1 = ((c_1t)**(1-sigma)-1)/(1-sigma)
u_c2 = ((c_2t)**(1-sigma)-1)/(1-sigma)

U = u_c1 + u_c2/(1+rho)
U

(c_1t**(1 - sigma) - 1)/(1 - sigma) + (c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1))

In [110]:
#solve the budget constraint for $c_{1t}$
budget_c1 = sm.solve(budget_c,c_1t)[0]
budget_c1

(-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1)

In [111]:
# We substitute budget_c1 into U_lifetime: 

U_lifetime = U.subs(c_1t, budget_c1)
U_lifetime

(c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1)) + (((-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1))**(1 - sigma) - 1)/(1 - sigma)

In [112]:
# Maximize lifetime utility

max_c2 = sm.diff(U_lifetime, c_2t)
max_c2 = sm.simplify(max_c2)
max_c2

(-c_2t+1*((-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1))**(1 - sigma)*(rho + 1) + c_2t+1**(1 - sigma)*(-c_2t+1 + r_t+1*w_t + w_t))/(c_2t+1*(rho + 1)*(-c_2t+1 + r_t+1*w_t + w_t))

In [113]:
# Lagrangian
budget_l = c_1t + c_2t/(1+r_t1) - w
L = U - lamb * budget_l
L



-lambda*(c_1t + c_2t+1/(r_t+1 + 1) - w_t) + (c_1t**(1 - sigma) - 1)/(1 - sigma) + (c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1))

In [117]:
def lambd(x):
        L_1 = sm.diff(L, x)
        L_2 = sm.Eq(L_1,0)
        L_3 = sm.solve(L_2, lamb) [0]
        return L_3

bb = lambd(c_1t)
gg = lambd(c_2t)

Eq_1 = sm.Eq(bb,gg)
Eq_2 = sm.solve(Eq_1, c_1t) [0]
Eq_3 = sm.Eq(c_1t,Eq_2)

Eq_4 = sm.solve(Eq_1, c_2t) [0]
Eq_5 = sm.Eq(c_2t,Eq_4)


c_1 = w-s
c_2 = (1+r_t1)*s
O_savings = Eq_3.subs(c_1t,c_1)
O_savings = O_savings.subs(c_2t,c_2)
Opt_savings = sm.solve(O_savings, s) [0]
Opt_savings


NotImplementedError: multiple generators [(rho*(r_t+1*s_t + s_t)**sigma/(r_t+1 + 1) + (r_t+1*s_t + s_t)**sigma/(r_t+1 + 1))**(1/sigma), s_t]
No algorithms are implemented to solve equation -s_t + w_t - ((s_t*(r_t+1 + 1))**sigma*(rho + 1)/(r_t+1 + 1))**(1/sigma)

In [5]:
ss_func = sm.lambdify((s,g,n,alpha,delta),kss)

## Numerical solution

Define the model **parameters**:

In [6]:
w = 1
rho = 0.2
r = 0.01

**Solve numerically** for the steady state:

In [7]:
def utility(rho):
    c_1 = w - c_2/(1+r)
    u(c_1) = ln(c_1)
    u(c_2) = ln(c_2)
    return u(c_1) + 1/(1+rho)*u(c_2)

analytical solution is: 1.904
 numerical solution is: 1.904


In [ ]:
# 1. grids
c1_vec = np.linspace(0,2,500)
c2_vec = np.linspace(0,2,500)
x1_grid,x2_grid = np.meshgrid(x1_vec,x2_vec,indexing='ij')
grid = _f(x1_grid,x2_grid)

# 2. main
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')
s = ax.plot_surface(x1_grid,x2_grid,grid, cmap=cm.jet)

# 3. name axis
ax.set_xlabel('x_1')
ax.set_ylabel('x_2')
ax.set_zlabel('f')

# 4. invert axis
ax.invert_xaxis()

# 5. add color
fig.colorbar(cs);

# Further analysis

ADD FURTHER ANALYSIS, VISUALIZATIONS AND EXTENSIONS.

# Conclusion

ADD CONCISE CONCLUSION.